In [ ]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [2]:
class GoogleCrawler():
    
    def __init__(self):
        self.url = 'https://www.google.com/search?q='    
    #  URL 萃取 From Google Search上 , using 第三方套件
    #  https://python-googlesearch.readthedocs.io/en/latest/
    def google_url_search_byOpenSource(query,tbs='qdr:m',num=10):
        array_url = [url for url in search('tsmc', tbs='qdr:m' , num=10)]
        return array_url
    # 網路擷取器
    def get_source(self,url):
        try:
            session = HTMLSession()
            response = session.get(url)
            return response
        except requests.exceptions.RequestException as e:
            print(e)
    # URL 萃取 From Google Search上
    def scrape_google(self,query):

        response = self.get_source(self.url + query)
        links = list(response.html.absolute_links)
        google_domains = ('https://www.google.', 
                          'https://google.', 
                          'https://webcache.googleusercontent.', 
                          'http://webcache.googleusercontent.', 
                          'https://policies.google.',
                          'https://support.google.',
                          'https://maps.google.')

        for url in links[:]:
            if url.startswith(google_domains):
                links.remove(url)
        return links
    
# URL萃取器，有link之外，也有標題
#     qdr:h (past hour)
#     qdr:d (past day)
#     qdr:w (past week)
#     qdr:m (past month)
#     qdr:y (past year)
    def google_search(self,query,timeline='',page='0'):
        url = self.url + query + '&tbs={timeline}&start={page}'.format(timeline=timeline,page=page)
        print('[Check][URL] URL : {url}'.format(url=url))
        response = self.get_source(self.url + query)
        return self.parse_googleResults(response)
    # Google Search Result Parsing
    def parse_googleResults(self,response):

        css_identifier_result = "tF2Cxc"
        css_identifier_title = "h3"
        css_identifier_link = "yuRUbf"
        css_identifier_text = "VwiC3b"
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.findAll("div", {"class": css_identifier_result})
        output = []
        for result in results:
            item = {
                'title': result.find(css_identifier_title).get_text(),
                'link': result.find("div", {"class": css_identifier_link}).find(href=True)['href'],
                'text': result.find("div", {"class": css_identifier_text}).get_text()
            }
            output.append(item)
        return output
    
    # 網頁解析器
    def html_parser(self,htmlText):
        soup = BeautifulSoup(htmlText, 'html.parser')
        return soup
    # 解析後，取<p>文字
    def html_getText(self,soup):
        orignal_text = ''
        for el in soup.find_all('p'):
            orignal_text += ''.join(el.find_all(text=True))
        return orignal_text
    
    def word_count(self, text):
        counts = dict()
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text)
        #words = text.replace(',','').split()
        for word in words:
            if word not in stop_words:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
        return counts
    def get_wordcount_json(self,whitelist , dict_data):
        data_array = []
        for i in whitelist:
            json_data = {
                'Date' : 'Week1',
                'Company' : i , 
                'Count' : dict_data[i]
            }
            data_array.append(json_data)
        return data_array
    def jsonarray_toexcel(self,data_array):
        df = pd.DataFrame(data=data_array)
        df.to_excel('result.xlsx' , index=False)
        return

## 程式使用情境
### 1. 用程式去呼叫Google網頁伺服器，然後將顯示出來的URL，抓出來。 (支援換頁及限縮搜尋時間)
### 2. 對我們要的URL，進行萃取，取得文字內容。
### 3. 將文字內容，進行word count計算。
### 4. 將word count結果儲存到Excel，以利分析之使用。

### 1. 在Google上輸入關鍵字，然後將顯示出來的URL，抓出來。 (支援換頁及限縮收尋時間)

#### 使用google_search 這支Function
- 參數使用 timeline : 搜尋時間 => 參數可以參考qdr:h (past hour), qdr:d (past day),qdr:w (past week),qdr:m (past month),qdr:y (past year)
- 參數使用 page : 換頁
```
def google_search(self,query,timeline='',page='0'):
    url = self.url + query + '&tbs={timeline}&start={page}'.format(timeline=timeline,page=page)
    print('[Check][URL] URL : {url}'.format(url=url))
    response = self.get_source(self.url + query)
    return self.parse_googleResults(response)
```
    
####  這邊用到get_source及parse_googleResults，兩支Function，為了解析Google Search網路資源，並將URL/標題/內文，找出來。
```
def get_source(self,url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)
def parse_googleResults(self,response):
        css_identifier_result = "tF2Cxc"
        css_identifier_title = "h3"
        css_identifier_link = "yuRUbf"
        css_identifier_text = "VwiC3b"
        soup = BeautifulSoup(response.text, 'html.parser')
        results = soup.findAll("div", {"class": css_identifier_result})
        output = []
        for result in results:
            item = {
                'title': result.find(css_identifier_title).get_text(),
                'link': result.find("div", {"class": css_identifier_link}).find(href=True)['href'],
                'text': result.find("div", {"class": css_identifier_text}).get_text()
            }
            output.append(item)
        return output ```

In [3]:
query = "TSMC ASML"
crawler = GoogleCrawler()
results = crawler.google_search(query , 'qdr:w' , '10')
print(results[:3])

[Check][URL] URL : https://www.google.com/search?q=TSMC ASML&tbs=qdr:w&start=10
[{'title': 'Better Semiconductor Stock: ASML vs. TSMC | The Motley Fool', 'link': 'https://www.fool.com/investing/2021/04/21/better-semiconductor-stock-asml-vs-tsmc/', 'text': "21 Apr 2021 — TSMC trades at 26 times forward earnings, which is much lower than ASML's forward P/E ratio of 45. TSMC's forward dividend yield of 1.5% is also\xa0..."}, {'title': 'Intel places ASML order to step up TSMC competition - Taipei ...', 'link': 'https://taipeitimes.com/News/biz/archives/2022/01/20/2003771688', 'text': "20 Jan 2022 — Intel Corp yesterday said it has placed its first order with ASML Holding NV to purchase the semiconductor industry's first TWINSCAN EXE:\xa0..."}, {'title': "ASML is the key to Intel's Resurrection Just like ASML helped...", 'link': 'https://semiwiki.com/semiconductor-services/semiconductor-advisors/302806-asml-is-the-key-to-intels-resurrection-just-like-asml-helped-tsmc-beat-intel/', 'text': "

### 2. 對我們要的URL，進行萃取，取得文字內容。
- 使用get_source 這支Function，取得網路資源後，
- 使用html_parser 這支Function，將網路資源進行解析。
- 使用html_getText 這支Function，將我要的區塊p tag的文字取出來。
```
def get_source(self,url):
    try:
        session = HTMLSession()
        response = session.get(url)
        return response
    except requests.exceptions.RequestException as e:
        print(e)
def html_parser(self,htmlText):
    soup = BeautifulSoup(htmlText, 'html.parser')
    return soup
def html_getText(self,soup):
    orignal_text = ''
    for el in soup.find_all('p'):
        orignal_text += ''.join(el.find_all(text=True))
    return orignal_text```

In [4]:
Target_URL = 'https://taipeitimes.com/News/biz/archives/2022/01/20/2003771688'
response = crawler.get_source(Target_URL)
soup = crawler.html_parser(response.text)
orignal_text = crawler.html_getText(soup)
print(orignal_text[:100])

Intel Corp yesterday said it has placed its first order with ASML Holding NV to purchase the semicon


### 3. 將文字內容，進行word count計算
#### 使用word_count function，將一篇文章內容，進行各文字的count個數
```
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def word_count(self, text):
        counts = dict()
        stop_words = set(stopwords.words('english'))
        words = word_tokenize(text)
        #words = text.replace(',','').split()
        for word in words:
            if word not in stop_words:
                if word in counts:
                    counts[word] += 1
                else:
                    counts[word] = 1
        return counts ```

In [5]:
result_wordcount = crawler.word_count(orignal_text)
result_wordcount

{'Intel': 5,
 'Corp': 1,
 'yesterday': 3,
 'said': 20,
 'placed': 1,
 'first': 7,
 'order': 2,
 'ASML': 7,
 'Holding': 1,
 'NV': 1,
 'purchase': 3,
 'semiconductor': 8,
 'industry': 6,
 '’': 17,
 'TWINSCAN': 5,
 'EXE': 1,
 ':': 6,
 '5200': 1,
 'system': 8,
 ',': 46,
 'US': 6,
 'chip': 4,
 'giant': 1,
 'aims': 2,
 'compete': 1,
 'Taiwan': 8,
 'Semiconductor': 5,
 'Manufacturing': 5,
 'Co': 4,
 '(': 9,
 'TSMC': 7,
 '台積電': 3,
 ')': 9,
 'advancing': 1,
 '2-nanometer': 4,
 'process': 1,
 'technology': 6,
 '.': 22,
 'The': 12,
 'Dutch': 1,
 'equipment': 1,
 'maker': 1,
 'EXE:5200': 2,
 'extreme': 1,
 'ultraviolet': 1,
 'EUV': 4,
 'high-volume': 1,
 'production': 3,
 'high': 2,
 'numerical': 1,
 'aperture': 1,
 'NA': 1,
 'produce': 2,
 '220': 1,
 'wafers': 2,
 'per': 1,
 'hour': 1,
 '150': 1,
 'previous': 1,
 'generation': 1,
 'EXE:5000': 2,
 'handle': 1,
 'launch': 1,
 'new': 5,
 '2024': 2,
 'Photo': 1,
 'ReutersASML': 1,
 'president': 3,
 'chief': 2,
 'officer': 2,
 'Martin': 1,
 'van': 1,


### 4. 將聲量結果儲存到Excel
#### 使用get_wordcount_json function濾掉不要的word count dict, 取自己要的(whitelist)
#### 同時利用jsonarray_toexcel function將結果，落地於Excel，以做聲量分析
```
def get_wordcount_json(self,whitelist , dict_data):
        data_array = []
        for i in whitelist:
            json_data = {
                'Date' : 'Week1',
                'Company' : i , 
                'Count' : dict_data[i]
            }
            data_array.append(json_data)
        return data_array
def jsonarray_toexcel(self,data_array):
    df = pd.DataFrame(data=data_array)
    df.to_excel('result.xlsx' , index=False)
    return
```

In [6]:
whitelist = ['ASML' , 'Intel']
end_result = crawler.get_wordcount_json(whitelist , result_wordcount)
print(end_result)
crawler.jsonarray_toexcel(end_result)
print('Excel is OK')

[{'Date': 'Week1', 'Company': 'ASML', 'Count': 7}, {'Date': 'Week1', 'Company': 'Intel', 'Count': 5}]
Excel is OK
